In [ ]:
pip install pandas numpy matplotlib seaborn scikit-learn


In [ ]:
from google.colab import files

# Upload the ratings.dat and movies.dat files
uploaded = files.upload()


Saving ratings.dat to ratings.dat
Saving movies.dat to movies.dat


In [ ]:
import pandas as pd

# Try loading the movies file with 'ISO-8859-1' encoding
movies = pd.read_csv("movies.dat", sep="::", engine="python", header=None, names=["movieId", "title", "genres"], encoding="ISO-8859-1")

# Check the data
print(movies.head())



   movieId                               title                        genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy


In [ ]:
# Load ratings.dat with 'ISO-8859-1' encoding
ratings = pd.read_csv("ratings.dat", sep="::", engine="python", header=None, names=["userId", "movieId", "rating", "timestamp"], encoding="ISO-8859-1")

# Check the data
print(ratings.head())


   userId  movieId  rating  timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


In [ ]:
# Merge the ratings and movies datasets on the 'movieId' column
merged_data = pd.merge(ratings, movies, on="movieId")

# Check the first few rows of the merged dataset
print(merged_data.head())


   userId  movieId  rating  timestamp                                   title  \
0       1     1193       5  978300760  One Flew Over the Cuckoo's Nest (1975)   
1       1      661       3  978302109        James and the Giant Peach (1996)   
2       1      914       3  978301968                     My Fair Lady (1964)   
3       1     3408       4  978300275                  Erin Brockovich (2000)   
4       1     2355       5  978824291                    Bug's Life, A (1998)   

                         genres  
0                         Drama  
1  Animation|Children's|Musical  
2               Musical|Romance  
3                         Drama  
4   Animation|Children's|Comedy  


In [ ]:
# Check basic info about the merged data
print(merged_data.info())

# Show summary statistics
print(merged_data.describe())

# Check number of unique users and movies
print(f"Unique users: {merged_data['userId'].nunique()}")
print(f"Unique movies: {merged_data['movieId'].nunique()}")

# Show the most popular movies based on the number of ratings
movie_counts = merged_data['title'].value_counts()
print(movie_counts.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1000209 non-null  int64 
 1   movieId    1000209 non-null  int64 
 2   rating     1000209 non-null  int64 
 3   timestamp  1000209 non-null  int64 
 4   title      1000209 non-null  object
 5   genres     1000209 non-null  object
dtypes: int64(4), object(2)
memory usage: 45.8+ MB
None
             userId       movieId        rating     timestamp
count  1.000209e+06  1.000209e+06  1.000209e+06  1.000209e+06
mean   3.024512e+03  1.865540e+03  3.581564e+00  9.722437e+08
std    1.728413e+03  1.096041e+03  1.117102e+00  1.215256e+07
min    1.000000e+00  1.000000e+00  1.000000e+00  9.567039e+08
25%    1.506000e+03  1.030000e+03  3.000000e+00  9.653026e+08
50%    3.070000e+03  1.835000e+03  4.000000e+00  9.730180e+08
75%    4.476000e+03  2.770000e+03  4.000000e+00  9.752209e+08
max    6

In [ ]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505171 sha256=cc0fefeaacdd1e88c0a0423791a46d3d721d47975cef3d3e80ad574d6a327e62
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader
from surprise import SVD, accuracy
from surprise.model_selection import train_test_split

# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_data[['userId', 'movieId', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Train a basic SVD (Singular Value Decomposition) model
model = SVD()
model.fit(trainset)

# Predict the ratings on the test set
predictions = model.test(testset)

# Compute the RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


RMSE: 0.8740
RMSE: 0.8739791661231519


In [ ]:
user_id = 67  # User ID
movie_id = 190  # Movie ID (e.g., the ID for "Star Wars: Episode IV")

# Predict the rating
prediction = model.predict(user_id, movie_id)
print(f"Predicted rating for user {user_id} on movie {movie_id}: {prediction.est}")


Predicted rating for user 67 on movie 190: 4.165367486035058


In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_data[['userId', 'movieId', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Train a basic SVD model
model = SVD()
model.fit(trainset)

# Predict the ratings on the test set
predictions = model.test(testset)

# Compute RMSE
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


RMSE: 0.8746
RMSE: 0.8746040195373288


In [ ]:
# Create a mapping of movieId to title
movie_indices = pd.Series(movies.index, index=movies['title']).to_dict()

# Define a function to get recommendations based on movie title
def content_based_recommender(title, cosine_sim=cosine_sim, movie_indices=movie_indices):
    # Get the index of the movie that matches the title
    idx = movie_indices.get(title)  # Use .get to avoid KeyError

    if idx is None:
        return "Movie not found in dataset"

    # Get the pairwise similarity scores for that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices_local = [i[0] for i in sim_scores]  # Renamed to avoid shadowing

    # Return the top 10 most similar movies
    return movies['title'].iloc[movie_indices_local]  # Use local variable here

# Test the content-based recommender
print(content_based_recommender("Star Wars: Episode IV - A New Hope (1977)"))


2036                                         Tron (1982)
2559    Star Wars: Episode I - The Phantom Menace (1999)
2104          Navigator: A Mediaeval Odyssey, The (1988)
2899                                 Time Bandits (1981)
1006                 20,000 Leagues Under the Sea (1954)
1698                                     Star Kid (1997)
2024                                 Return to Oz (1985)
1952                                         Dune (1984)
1985                     Honey, I Shrunk the Kids (1989)
171                                   Judge Dredd (1995)
Name: title, dtype: object


In [ ]:
# Function to get hybrid recommendations
def hybrid_recommender(user_id, movie_id, top_n=10):
    # Get collaborative recommendations (from SVD model)
    collab_pred = model.predict(user_id, movie_id)

    # Get content-based recommendations
    movie_title = movies[movies['movieId'] == movie_id]['title'].values[0]
    content_recs = content_based_recommender(movie_title)

    # Combine the recommendations
    print(f"Collaborative prediction: {collab_pred.est}")
    print(f"Top 10 Content-Based Recommendations:")
    for movie in content_recs:
        print(movie)

# Example usage: hybrid recommender for user 1 and movie 100
hybrid_recommender(1, 100)


Collaborative prediction: 3.5734606487564147
Top 10 Content-Based Recommendations:
Eye for an Eye (1996)
Juror, The (1996)
Mary Reilly (1996)
City Hall (1996)
Taxi Driver (1976)
Babysitter, The (1995)
Disclosure (1994)
Death and the Maiden (1994)
Dolores Claiborne (1994)
Murder in the First (1995)
